In [11]:
import pandas as pd
import numpy as np

In [37]:
prediction = pd.read_csv('results/prediction_sqf_rnn_task15.csv').set_index(['TIMESTAMP', 'ZONEID'])

actual = pd.read_csv('data/Task 16/train16.csv', parse_dates=['TIMESTAMP']).set_index('TIMESTAMP')['2014-06-01 01:00':]\
    .reset_index().set_index(['TIMESTAMP', 'ZONEID'])

,,POWER
TIMESTAMP,ZONEID,
2014-06-01 01:00:00,1,0.068526
2014-06-01 02:00:00,1,0.067115
2014-06-01 03:00:00,1,0.047308
2014-06-01 04:00:00,1,0.030449
2014-06-01 05:00:00,1,0.053269
...,...,...
2014-06-30 20:00:00,3,0.000000
2014-06-30 21:00:00,3,0.000025
2014-06-30 22:00:00,3,0.084900


In [49]:
def sample(quantiles, n):
    ps = np.random.choice(a=[str(p / 100) for p in range(1, 100)], size=n, replace=True)
    xs = quantiles[ps]
    return np.array(xs)[None,...]

sampled = sample(prediction.iloc[0], 20)

In [115]:
def energy_score(quantiles, y, n=100):
    """
    Calculates the energy score.
    :param quantiles: np array (99,)
    :param y: (d,)
    :param n: n
    :return:
    """
    xs = sample(quantiles, n)
    s1 = np.mean(np.linalg.norm(xs - y, axis=0))
    s2 = - 1 / 2 * np.mean(np.linalg.norm(xs[None, ...] - xs[None, ...].T, axis=1))
    return s1 + s2

# energy_score for first prediction point
energy_score(prediction.iloc[0], np.array(actual.iloc[0]))

0.0356851832472436